In [4]:
from datetime import datetime
import pandas as pd
import numpy as np
import os
import sys
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from functools import partial

parent_directory = os.path.dirname(os.getcwd())
sys.path.append(parent_directory)  

from utils.utils import load_data, parse_duration, determine_time_window, prestation_duration, convert_duration, define_client_time_matrix, define_inter_time_matrix, create_full_time_matrix

path = "/Users/etiennedufayet/Desktop/Cours/X4A/data_4_health/data/"
data_file = "ChallengeXHEC.xlsx"


In [5]:
data = load_data()

In [7]:
df = data['JAN24']
df_dispo = data['dispo']
df_intervenants = data['intervenants']
intervenant_competence = {}
intervenant_driving_license = {}

for intervenant_id, competences in zip(df_intervenants['ID Intervenant'],df_intervenants["Compétences"]): 
    competences = competences.split(', ')
    if 'TOILETTE' in competences:
        competences.append('TOILETTE_MAT')
        competences.append('TOILETTE_SOIR')
    if 'REPAS' in competences:
        competences.append('PDJ')
        competences.append('DIN')
        competences.append('DEJ')
    intervenant_competence[intervenant_id]=competences

for intervenant_id, driving_license in zip(df_intervenants['ID Intervenant'],df_intervenants["Véhicule personnel"]): 
    intervenant_driving_license[intervenant_id] = (1 if driving_license == 'Oui' else 0)

In [8]:
for i in range(len(df)):
    heure_debut = df.at[i, 'Heure de début']
    if df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 10:
        df.at[i, 'Prestation'] = 'PDJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS' and heure_debut.hour < 14 and heure_debut.hour>12 :
        df.at[i, 'Prestation'] = 'DEJ'
    
    elif df.at[i, 'Prestation'] == 'REPAS':
        df.at[i, 'Prestation'] = 'DIN'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour < 12:
        df.at[i, 'Prestation'] = 'TOILETTE_MAT'
    
    elif df.at[i, 'Prestation'] == 'TOILETTE' and heure_debut.hour > 12:
        df.at[i, 'Prestation'] = 'TOILETTE_SOIR'

In [9]:
df['time_window'] = df['Prestation'].apply(determine_time_window)

prestation_duration_list = []
for i in range(len(df)):
    h1 = df.iloc[i].loc['Heure de début']
    h2 = df.iloc[i].loc['Heure de fin']

    prestation_duration_list.append(prestation_duration(h1,h2))

df['Prestation_duration'] = prestation_duration_list

In [10]:
c2c_dist = pd.read_csv("/Users/etiennedufayet/Desktop/Cours/X4A/data_4_health/data/distance_matrix_client_to_client.csv")
i2c_dist = pd.read_csv("/Users/etiennedufayet/Desktop/Cours/X4A/data_4_health/data/distance_matrix_inter_to_client.csv")

list_clients_1 = c2c_dist["ID Client 1"].unique()
list_clients_2 = c2c_dist["ID Client 2"].unique()
full_client_list = list(set([*list_clients_1, *list_clients_2]))
nb_full_client = len(full_client_list)

In [11]:
client_id_to_idx = {}
client_idx_to_id = {}
for k in range(len(full_client_list)):
    client_id_to_idx[full_client_list[k]] = k
    client_idx_to_id[k] = full_client_list[k]

In [13]:
client_client_matrix_car = define_client_time_matrix(c2c_dist, "car", client_id_to_idx)
client_client_matrix_bike = define_client_time_matrix(c2c_dist, "bike", client_id_to_idx)

In [14]:
list_inter_columns = i2c_dist['ID Intervenant']
full_inter_list = list_inter_columns.unique()

inter_id_to_idx = {}
inter_idx_to_id = {}
for k in range(len(full_inter_list)):
    inter_id_to_idx[full_inter_list[k]] = k
    inter_idx_to_id[k] = full_inter_list[k]

In [16]:
inter_time_matrix_car = define_inter_time_matrix(
    i2c_dist, full_client_list, inter_id_to_idx, client_id_to_idx, vehicle_type="Car"
)
inter_time_matrix_bike = define_inter_time_matrix(
    i2c_dist, full_client_list, vehicle_type="Bike"
)

In [18]:
full_time_matrix_car = create_full_time_matrix(
    client_client_matrix_car, inter_time_matrix_car
)
full_time_matrix_bike = create_full_time_matrix(
    client_client_matrix_bike, inter_time_matrix_bike
)

In [19]:
list_skills = df["Prestation"].unique()

def specific_to_date(date):
    date = "2024-01-01"

    condition = df["Date"] == date
    c_l_test = df[condition]
    c_l_test.drop(columns=["ID Intervenant", "Date", "Heure de début", "Heure de fin"], inplace=True)
    c_l_test

    parts = date.strip().split("-")

    # Extract the year, month, and day as integers
    year = int(parts[0])
    month = int(parts[1])
    day = int(parts[2])
    
    available_inter_day = df_dispo[df_dispo[datetime(year, month, day, 00, 00, 00)]==1]['ID Intervenant'].to_list()
    
    return(c_l_test, available_inter_day)

In [107]:
def create_data_model(
    df_client,
    full_time_matrix_car,
    full_time_matrix_bike,
    list_available_inter,
    client_id_to_idx=client_id_to_idx,
    inter_id_to_idx=inter_id_to_idx,
    nb_full_client=nb_full_client,
):
    nb_clients = len(df_client)
    nb_inter = len(list_available_inter)
    data = {}

    # Initialization
    data["time_matrix_car"] = [
        [10000 for i in range(nb_clients + nb_inter)] for k in range(nb_clients + nb_inter)
    ]
    data["time_matrix_bike"] = [
        [10000 for i in range(nb_clients + nb_inter)] for k in range(nb_clients + nb_inter)
    ]
    data["service_times"] = [0 for i in range(nb_clients + nb_inter)]
    data["time_windows"] = [(0, 1440) for i in range(nb_clients + nb_inter)]

    data["num_vehicles"] = nb_inter
    data["is_driver"] = [
        intervenant_driving_license[list_available_inter[i]] for i in range(nb_inter)
    ]  
    data["starts"] = [nb_clients + i for i in range(nb_inter)]
    data["ends"] = [nb_clients + i for i in range(nb_inter)]

    data["inter_skills"] = [
        intervenant_competence[list_available_inter[i]] for i in range(nb_inter)
    ]  
    data["node_requirements"] = [None for i in range(nb_clients + nb_inter)]

    data['drivers'] = {i: { 0 if i == 0 else 1} for i in range(nb_inter)}


    
    for i in range(nb_clients):
        need_id_client = df_client["ID Client"][i]
        need_idx_client = client_id_to_idx[need_id_client]

        for j in range(nb_clients):
            other_id_client = df_client["ID Client"][j]
            other_idx_client = client_id_to_idx[other_id_client]

            travel_time_car = full_time_matrix_car[need_idx_client, other_idx_client]
            travel_time_bike = full_time_matrix_bike[need_idx_client, other_idx_client]
            data["time_matrix_car"][i][j] = travel_time_car
            data["time_matrix_bike"][i][j] = travel_time_bike

        for j in range(nb_inter):
            id_inter = list_available_inter[j]
            idx_inter = inter_id_to_idx[id_inter]

            travel_time_car = full_time_matrix_car[need_idx_client][
                nb_full_client + idx_inter
            ]
            travel_time_bike = full_time_matrix_bike[need_idx_client][
                nb_full_client + idx_inter
            ]

            data["time_matrix_car"][i][nb_clients + j] = travel_time_car
            data["time_matrix_car"][nb_clients + j][i] = data["time_matrix_car"][i][
                nb_clients + j
            ]
            data["time_matrix_bike"][i][nb_clients + j] = travel_time_bike
            data["time_matrix_bike"][nb_clients + j][i] = data["time_matrix_bike"][i][
                nb_clients + j
            ]

        service_time = int(df_client["Prestation_duration"][i])
        data["service_times"][i] = service_time

        time_window_temp = df_client["time_window"][i]
        
        time_window_start = int(float(time_window_temp[0]) * 60)
        time_window_end = int(float(time_window_temp[1]) * 60)
        data["time_windows"][i] = (time_window_start, time_window_end)

        skills_required = df_client["Prestation"][i]
        data["node_requirements"][i] = skills_required

    for i in range(nb_clients + nb_inter):
        data["time_matrix_car"][i][i] = 0
        data["time_matrix_bike"][i][i] = 0

    data["time_windows_with_service"] = [
        (
            data["time_windows"][i][0] + data["service_times"][i],
            data["time_windows"][i][1] + data["service_times"][i],
        )
        for i in range(len(data["service_times"]))
    ]

    return data

In [93]:
def print_detailed_solution_real_data(
    data, manager, routing, solution, inter_idx_to_id=inter_idx_to_id, client_idx_to_id=client_idx_to_id
):
    """Prints detailed solution on console, including service and wait times."""
    print(f"Objective: {solution.ObjectiveValue()}")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_time = 0

    day_missions = []
    for vehicle_id in range(data["num_vehicles"]):

        
        index = routing.Start(vehicle_id)
        inter_id = inter_idx_to_id[vehicle_id]

        print(f"Route for vehicle {inter_id}:")
        print(f"Permis: {intervenant_driving_license[inter_id]==1}")
        worked_time = 0

        vehicle_missions = []
        i=0
        

        while not routing.IsEnd(index):


            time_var = time_dimension.CumulVar(index)
            next_index = solution.Value(routing.NextVar(index))
            next_node = manager.IndexToNode(next_index)

            service_time = data["service_times"][manager.IndexToNode(index)]

            # Arrival time at current location
            arrival_time = solution.Min(time_var) - service_time

            # End of service time = arrival time + service time at current location
            end_of_service_time = solution.Min(time_var)

            # Initialization
            departure_time = end_of_service_time

            if next_index != routing.End(vehicle_id):
                next_arrival_time = solution.Min(time_dimension.CumulVar(next_index))
                if data["is_driver"][vehicle_id]:
                    next_travel_time = data["time_matrix_car"][
                        manager.IndexToNode(index)
                    ][next_node]

                else:
                    next_travel_time = data["time_matrix_bike"][
                        manager.IndexToNode(index)
                    ][next_node]
                next_service_time = data["service_times"][next_node]
                departure_time = (
                    next_arrival_time - next_travel_time - next_service_time
                )


            break_time = departure_time - end_of_service_time

            if next_index == routing.End(vehicle_id):
                break_time = 0
                departure_time = end_of_service_time

            arrival_time_str = str(arrival_time // 60).zfill(2) + " : " + str(arrival_time % 60).zfill(2)
            end_of_service_time_str = (
                str(end_of_service_time // 60).zfill(2) + " : " + str(end_of_service_time % 60).zfill(2)
            )
            break_time_str = str(break_time // 60).zfill(2) + " : " + str(break_time % 60).zfill(2)
            departure_time_str = (
                str(departure_time // 60).zfill(2) + " : " + str(departure_time % 60).zfill(2)
            )

            """print(
                f"Location {manager.IndexToNode(index) + 1} (Arrival: {arrival_time_str}, "
                f" (Skills required: {skills_required}), End of service time {end_of_service_time_str}, Break: {break_time_str}, "
                f"Departure: {departure_time_str}) -> "
            )"""

            if i>0: 
                vehicle_missions.append([client_idx_to_id[manager.IndexToNode(index) + 1], inter_id, arrival_time_str, end_of_service_time_str])           

            if break_time > 30: 
                worked_time -= break_time
 
            index = next_index
        
            i += 1

        # Handle final location's arrival time
        final_arrival_time = solution.Min(time_dimension.CumulVar(index))
        print(
            f"Location {manager.IndexToNode(index) + 1} (Arrival: {final_arrival_time})"
        )
        worked_time += final_arrival_time
        print(worked_time)
        
        total_time += worked_time
        print("\n")

        day_missions.append(vehicle_missions)
    print(f"Total time of all routes: {total_time}min")
    
    return(day_missions)

In [104]:
def main_real_data(
    c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
):
    """Solve the VRP with time windows from specified locations."""
    # Instantiate the data problem.
    data = create_data_model(
        c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list
    )

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["time_matrix_car"]), data["num_vehicles"], data["starts"], data["ends"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index, type):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        service_time = data["service_times"][to_node]
        if type == 1:  
            return data["time_matrix_car"][from_node][to_node] + service_time
        else:
            return data["time_matrix_bike"][from_node][to_node] + service_time

    car_callback = partial(time_callback, type=1)
    bike_callback = partial(time_callback, type=0)
    

    car_index = routing.RegisterTransitCallback(car_callback)
    bike_index = routing.RegisterTransitCallback(bike_callback)

    routing.SetArcCostEvaluatorOfVehicle(car_index, 1)  
    routing.SetArcCostEvaluatorOfVehicle(bike_index, 0)

    """list_index = []
    for k in range(data["num_vehicles"]):
        if data["is_driver"][k] == 1:
            routing.SetArcCostEvaluatorOfVehicle(car_index, k)
            list_index.append(car_index)
        else:
            routing.SetArcCostEvaluatorOfVehicle(bike_index, k)
            list_index.append(bike_index)"""
            
            # Enforce node requirements based on vehicle skills
    for node_idx, requirement in enumerate(data["node_requirements"]):
        if requirement:  # Check if the node has specific skill requirements
            allowed_vehicles = [
                vehicle_id
                for vehicle_id, skills in enumerate(data["inter_skills"])
                if requirement in skills
            ]
            manager_idx = manager.NodeToIndex(node_idx)
            if manager_idx >= 0:  # Ensure manager index is valid
                routing.SetAllowedVehiclesForIndex(allowed_vehicles, manager_idx)

    # Add Time constraint.
    dimension_name = "Time"
    routing.AddDimensionWithVehicleTransitAndCapacity(
        [bike_index for i in range(data['num_vehicles'])], 
        10000,  # no slack
        [10000 for i in range(data["num_vehicles"])],  # vehicle maximum travel time
        True,  # start cumul to zero
        dimension_name,
    )

    time_dimension = routing.GetDimensionOrDie(dimension_name)

    # Add time window constraints for each location except start and end.
    for location_idx, time_window in enumerate(data["time_windows_with_service"]):
        if location_idx in data["starts"]:
            continue
        if location_idx in data["ends"]:
            continue
        index = manager.NodeToIndex(location_idx)
        start_time_window = time_window[0]
        end_time_window = time_window[1]
        time_dimension.CumulVar(index).SetRange(start_time_window, end_time_window)

    # Instantiate route start and end times to produce feasible times.
    for i in range(data["num_vehicles"]):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i))
        )
        routing.AddVariableMinimizedByFinalizer(time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.log_search = True
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    """search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = 10
"""
    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        # print_solution(data, manager, routing, solution)
        return(print_detailed_solution_real_data(data, manager, routing, solution))
    else: 
        print('No SOL')

In [86]:
total_missions = []
for date in df['Date'].unique():
    print(date)
    c_l_test, available_inter_day = specific_to_date(date)
    missions_date = main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, full_inter_list)
    for mission_date in missions_date:
        if mission_date:  
            for mission in mission_date:
                mission.append(date)  
                total_missions.append(mission)  

print(total_missions)

2024-01-01 00:00:00
Objective: 0
Route for vehicle 162858075:
Permis: True
Location 67 (Arrival: 0)
0


Route for vehicle 78007018:
Permis: True
Location 68 (Arrival: 0)
0


Route for vehicle 856089133:
Permis: False
Location 69 (Arrival: 0)
0


Route for vehicle 842621873:
Permis: True
Location 70 (Arrival: 0)
0


Route for vehicle 838320706:
Permis: True
Location 71 (Arrival: 0)
0


Route for vehicle 776399834:
Permis: True
Location 72 (Arrival: 0)
0


Route for vehicle 854577575:
Permis: False
Location 73 (Arrival: 0)
0


Route for vehicle 648993440:
Permis: True
Location 74 (Arrival: 0)
0


Route for vehicle 888888888:
Permis: False
Location 75 (Arrival: 670)
254


Route for vehicle 480302361:
Permis: True
Location 76 (Arrival: 624)
220


Route for vehicle 710283561:
Permis: True
Location 77 (Arrival: 661)
252


Route for vehicle 810259688:
Permis: False
Location 78 (Arrival: 1210)
350


Route for vehicle 764144581:
Permis: True
Location 79 (Arrival: 1284)
383


Route for vehicle 7

/var/folders/p7/1dmg12pd09b9zpy13gpy_yn00000gn/T/ipykernel_16409/541601533.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["ID Intervenant", "Date", "Heure de début", "Heure de fin"], inplace=True)
I0000 00:00:1710367564.108881 4813438 search.cc:282] Start search (memory used = 68.83 MB)
I0000 00:00:1710367564.110031 4813438 search.cc:282] Root node processed (time = 0 ms, constraints = 731, memory used = 69.14 MB)
/var/folders/p7/1dmg12pd09b9zpy13gpy_yn00000gn/T/ipykernel_16409/541601533.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["ID Intervenant", "Date", "Heu

Objective: 0
Route for vehicle 162858075:
Permis: True
Location 67 (Arrival: 0)
0


Route for vehicle 78007018:
Permis: True
Location 68 (Arrival: 0)
0


Route for vehicle 856089133:
Permis: False
Location 69 (Arrival: 0)
0


Route for vehicle 842621873:
Permis: True
Location 70 (Arrival: 0)
0


Route for vehicle 838320706:
Permis: True
Location 71 (Arrival: 0)
0


Route for vehicle 776399834:
Permis: True
Location 72 (Arrival: 0)
0


Route for vehicle 854577575:
Permis: False
Location 73 (Arrival: 0)
0


Route for vehicle 648993440:
Permis: True
Location 74 (Arrival: 0)
0


Route for vehicle 888888888:
Permis: False
Location 75 (Arrival: 670)
254


Route for vehicle 480302361:
Permis: True
Location 76 (Arrival: 624)
220


Route for vehicle 710283561:
Permis: True
Location 77 (Arrival: 661)
252


Route for vehicle 810259688:
Permis: False
Location 78 (Arrival: 1210)
350


Route for vehicle 764144581:
Permis: True
Location 79 (Arrival: 1284)
383


Route for vehicle 78007874:
Permis: Tru

I0000 00:00:1710367564.313221 4813438 search.cc:282] Start search (memory used = 84.14 MB)
/var/folders/p7/1dmg12pd09b9zpy13gpy_yn00000gn/T/ipykernel_16409/541601533.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["ID Intervenant", "Date", "Heure de début", "Heure de fin"], inplace=True)
I0000 00:00:1710367564.313382 4813438 search.cc:282] Root node processed (time = 0 ms, constraints = 731, memory used = 84.14 MB)
I0000 00:00:1710367564.316269 4813438 search.cc:282] Solution #0 (0, time = 3 ms, branches = 58, failures = 1, depth = 33, memory used = 84.20 MB)
I0000 00:00:1710367564.316326 4813438 search.cc:282] Finished search tree (time = 3 ms, branches = 58, failures = 35, memory used = 84.22 MB)
I0000 00:00:1710367564.316390 4813438 search.cc:282] End search (time = 3 ms, 

Location 81 (Arrival: 1271)
294


Route for vehicle 1452747150:
Permis: False
Location 82 (Arrival: 0)
0


Route for vehicle 78005437:
Permis: True
Location 83 (Arrival: 1319)
443


Route for vehicle 213237245:
Permis: True
Location 84 (Arrival: 1335)
245


Route for vehicle 609468992:
Permis: True
Location 85 (Arrival: 1312)
241


Route for vehicle 818696864:
Permis: True
Location 86 (Arrival: 1271)
137


Route for vehicle 813991780:
Permis: True
Location 87 (Arrival: 1293)
180


Route for vehicle 739888851:
Permis: True
Location 88 (Arrival: 0)
0


Route for vehicle 78012267:
Permis: True
Location 89 (Arrival: 0)
0


Route for vehicle 700168298:
Permis: True
Location 90 (Arrival: 1348)
333


Total time of all routes: 3332min
2024-01-26 00:00:00
Objective: 0
Route for vehicle 162858075:
Permis: True
Location 67 (Arrival: 0)
0


Route for vehicle 78007018:
Permis: True
Location 68 (Arrival: 0)
0


Route for vehicle 856089133:
Permis: False
Location 69 (Arrival: 0)
0


Route for vehicle

I0000 00:00:1710367564.579655 4813438 search.cc:282] Start search (memory used = 94.56 MB)
/var/folders/p7/1dmg12pd09b9zpy13gpy_yn00000gn/T/ipykernel_16409/541601533.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  c_l_test.drop(columns=["ID Intervenant", "Date", "Heure de début", "Heure de fin"], inplace=True)
I0000 00:00:1710367564.579851 4813438 search.cc:282] Root node processed (time = 0 ms, constraints = 731, memory used = 94.59 MB)
I0000 00:00:1710367564.582518 4813438 search.cc:282] Solution #0 (0, time = 2 ms, branches = 58, failures = 1, depth = 33, memory used = 94.91 MB)
I0000 00:00:1710367564.582564 4813438 search.cc:282] Finished search tree (time = 2 ms, branches = 58, failures = 35, memory used = 94.94 MB)
I0000 00:00:1710367564.582623 4813438 search.cc:282] End search (time = 2 ms, 

In [103]:
a = main_real_data(c_l_test, full_time_matrix_car, full_time_matrix_bike, available_inter_day)

I0000 00:00:1710368492.558599 4813438 search.cc:282] Start search (memory used = 50.70 MB)
I0000 00:00:1710368492.558772 4813438 search.cc:282] Root node processed (time = 0 ms, constraints = 610, memory used = 50.81 MB)


Objective: 0
Route for vehicle 162858075:
Permis: True
Location 67 (Arrival: 570)
172


Route for vehicle 78007018:
Permis: True
Location 68 (Arrival: 0)
0


Route for vehicle 856089133:
Permis: False
Location 69 (Arrival: 480)
82


Route for vehicle 842621873:
Permis: True
Location 70 (Arrival: 1200)
173


Route for vehicle 838320706:
Permis: True
Location 71 (Arrival: 1200)
137


Route for vehicle 776399834:
Permis: True
Location 72 (Arrival: 1200)
219


Route for vehicle 854577575:
Permis: False
Location 73 (Arrival: 1200)
154


Route for vehicle 648993440:
Permis: True
Location 74 (Arrival: 1200)
161


Route for vehicle 888888888:
Permis: False
Location 75 (Arrival: 1200)
159


Route for vehicle 480302361:
Permis: True
Location 76 (Arrival: 1200)
102


Route for vehicle 710283561:
Permis: True
Location 77 (Arrival: 0)
0


Route for vehicle 810259688:
Permis: False
Location 78 (Arrival: 1170)
92


Route for vehicle 764144581:
Permis: True
Location 79 (Arrival: 1170)
31


Total time 

I0000 00:00:1710368496.098132 4813438 search.cc:282] Solution #0 (159, time = 3539 ms, branches = 1110600, failures = 555240, depth = 33, memory used = 53.00 MB)
I0000 00:00:1710368496.098509 4813438 search.cc:282] Solution #1 (19, maximum = 159, time = 3539 ms, branches = 1110618, failures = 555243, depth = 33, Relocate<1>, neighbors = 1, filtered neighbors = 1, accepted neighbors = 1, memory used = 53.17 MB)
I0000 00:00:1710368496.098713 4813438 search.cc:282] Solution #2 (0, maximum = 159, time = 3540 ms, branches = 1110637, failures = 555246, depth = 33, Relocate<1>, neighbors = 8, filtered neighbors = 2, accepted neighbors = 2, memory used = 53.19 MB)
I0000 00:00:1710368496.098748 4813438 search.cc:282] Finished search tree (time = 3540 ms, branches = 1110637, failures = 555279, neighbors = 8, filtered neighbors = 2, accepted neigbors = 2, memory used = 53.19 MB)
I0000 00:00:1710368496.098807 4813438 search.cc:282] End search (time = 3540 ms, branches = 1110637, failures = 555279,

In [87]:
for missions_day in total_missions:
    for mission in missions_day:
        print(mission)

740412946
888888888
07 : 00
08 : 35
2024-01-01 00:00:00
78691389
888888888
08 : 37
11 : 07
2024-01-01 00:00:00
860937824
480302361
07 : 00
07 : 45
2024-01-01 00:00:00
315244686
480302361
07 : 54
08 : 54
2024-01-01 00:00:00
432156741
480302361
09 : 04
10 : 19
2024-01-01 00:00:00
78690893
710283561
07 : 00
07 : 45
2024-01-01 00:00:00
826666511
710283561
07 : 55
08 : 40
2024-01-01 00:00:00
1452763176
710283561
08 : 46
09 : 46
2024-01-01 00:00:00
756396600
710283561
10 : 00
11 : 00
2024-01-01 00:00:00
824480809
810259688
07 : 00
07 : 45
2024-01-01 00:00:00
82135699
810259688
07 : 49
08 : 34
2024-01-01 00:00:00
1453084257
810259688
08 : 36
09 : 21
2024-01-01 00:00:00
648827964
810259688
09 : 31
10 : 16
2024-01-01 00:00:00
859852985
810259688
18 : 00
20 : 00
2024-01-01 00:00:00
570590313
764144581
07 : 00
07 : 30
2024-01-01 00:00:00
741595162
764144581
07 : 37
08 : 07
2024-01-01 00:00:00
727679560
764144581
08 : 07
08 : 37
2024-01-01 00:00:00
78691334
764144581
08 : 38
09 : 08
2024-01-01 00:

In [89]:
total_missions

[[740412946,
  888888888,
  '07 : 00',
  '08 : 35',
  Timestamp('2024-01-01 00:00:00')],
 [78691389, 888888888, '08 : 37', '11 : 07', Timestamp('2024-01-01 00:00:00')],
 [860937824,
  480302361,
  '07 : 00',
  '07 : 45',
  Timestamp('2024-01-01 00:00:00')],
 [315244686,
  480302361,
  '07 : 54',
  '08 : 54',
  Timestamp('2024-01-01 00:00:00')],
 [432156741,
  480302361,
  '09 : 04',
  '10 : 19',
  Timestamp('2024-01-01 00:00:00')],
 [78690893, 710283561, '07 : 00', '07 : 45', Timestamp('2024-01-01 00:00:00')],
 [826666511,
  710283561,
  '07 : 55',
  '08 : 40',
  Timestamp('2024-01-01 00:00:00')],
 [1452763176,
  710283561,
  '08 : 46',
  '09 : 46',
  Timestamp('2024-01-01 00:00:00')],
 [756396600,
  710283561,
  '10 : 00',
  '11 : 00',
  Timestamp('2024-01-01 00:00:00')],
 [824480809,
  810259688,
  '07 : 00',
  '07 : 45',
  Timestamp('2024-01-01 00:00:00')],
 [82135699, 810259688, '07 : 49', '08 : 34', Timestamp('2024-01-01 00:00:00')],
 [1453084257,
  810259688,
  '08 : 36',
  '09 :

In [92]:
# Transformer chaque Timestamp en chaîne de caractères représentant la date, ou le laisser tel quel si cela convient à vos besoins
donnees_modifiees = [[e.strftime('%Y-%m-%d') if isinstance(e, pd.Timestamp) else e for e in sous_liste] for sous_liste in total_missions]

# Créer un DataFrame à partir des données modifiées
colonnes = ["ID Client", "ID Intervenant", "Heure de début", "Heure de fin", "Date"]
df_final = pd.DataFrame(donnees_modifiees, columns=colonnes)

df_final.to_csv('../data/optimized_missions.csv')